In [8]:
import pandas as pd 
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
from __future__ import print_function
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
from sklearn import preprocessing 

import cv2
import skimage.io as io

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [9]:
pd.set_option('display.max_rows', None)
train1=pd.read_csv("train.csv")
# print(train1.shape)
test1 = pd.read_csv('test.csv')
# print(test2.shape)

y_=train1.pop("label")
y1 = np.zeros((len(y_), 10))  
y1[np.arange(len(y_)), y_] = 1

train2=train1.values
train2 = train2/255
train2=train2.reshape((42000,28,28))
train2=np.expand_dims(train2,axis=-1)

test2=test1.values
test2 = test2/255
test2=test2.reshape((28000,28,28))
test=np.expand_dims(test2,axis=-1)



print(type(train2))
print(train2.shape)
print(test.shape)
print(y1.shape)

from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid=train_test_split(train2,y1,test_size=0.2,random_state=1)

<class 'numpy.ndarray'>
(42000, 28, 28, 1)
(28000, 28, 28, 1)
(42000, 10)


In [10]:
model_name = 'classic_CNN'
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(MaxPooling2D((1, 1)))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))

model.add(Dense(10))
model.add(BatchNormalization())
model.add(Activation(activation='softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_8 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
activation_8 (Activation)    (None, 28, 28, 64)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
batch_normalization_9 (Batch (None, 28, 28, 128)       512       
_________________________________________________________________
activation_9 (Activation)    (None, 28, 28, 128)       0         
__________

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='constant',
    cval=0)

optimizer = keras.optimizers.Adam(lr=10e-6)

model_path = './saved_models/{}.h5'.format(model_name)
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])
# Fit the model on the batches generated by datagen.flow().
batch_size = 16
aug_ratio = 1
epochs = 25
steps_per_epoch = int(aug_ratio * X_train.shape[0] / batch_size)
validation_steps = int(aug_ratio * X_valid.shape[0] / batch_size)
model_history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = batch_size),
                                    epochs = epochs,
                                    validation_data = (X_valid, y_valid),
                                    callbacks = [checkpoint, earlystop],
                                    steps_per_epoch=steps_per_epoch,
                                    validation_steps=validation_steps)

Epoch 1/25
2098/2100 [============================>.] - ETA: 0s - loss: 1.6423 - acc: 0.4707
Epoch 00001: val_loss improved from inf to 0.82665, saving model to ./saved_models/classic_CNN.h5
2100/2100 [==============================] - 41s 20ms/step - loss: 1.6418 - acc: 0.4710 - val_loss: 0.8266 - val_acc: 0.8283
Epoch 2/25
2099/2100 [============================>.] - ETA: 0s - loss: 1.1358 - acc: 0.7018
Epoch 00002: val_loss improved from 0.82665 to 0.61789, saving model to ./saved_models/classic_CNN.h5
2100/2100 [==============================] - 40s 19ms/step - loss: 1.1358 - acc: 0.7018 - val_loss: 0.6179 - val_acc: 0.8957
Epoch 3/25
2098/2100 [============================>.] - ETA: 0s - loss: 0.9777 - acc: 0.7677
Epoch 00003: val_loss improved from 0.61789 to 0.52190, saving model to ./saved_models/classic_CNN.h5
2100/2100 [==============================] - 41s 19ms/step - loss: 0.9777 - acc: 0.7678 - val_loss: 0.5219 - val_acc: 0.9256
Epoch 4/25
2098/2100 [======================

In [ ]:
training_loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

plt.plot(training_loss, label="training_loss")
plt.plot(val_loss, label="validation_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Learning Curve")
plt.legend(loc='best')
plt.show()

In [ ]:
training_acc = model_history.history['acc']
val_acc = model_history.history['val_acc']

plt.plot(training_acc, label="training_acc")
plt.plot(val_acc, label="validation_acc")
plt.xlabel("Epochs")
plt.ylabel("Acc")
plt.title("Learning Curve")
plt.legend(loc='best')
plt.show()

In [ ]:
# X_test, X_id = load_test_data()
# test1 = pd.read_csv('/data/examples/may_the_4_be_with_u/where_am_i/img-submission.csv')

model_path = './saved_models/{}.h5'.format(model_name)
model = load_model(model_path)

scores = model.evaluate(X_valid, y_valid, verbose=1)
print('Validation loss:', scores[0])
print('Validation accuracy:', scores[1])

y_test_pred = model.predict_classes(test)
y_test_pred_df = pd.DataFrame({'ImageId': np.array(test1.index+1), 'Label':y_test_pred})
y_test_pred_df.to_csv('./submissions/{}.csv'.format(model_name), index=False)